## Evaluating the solution 💯

We have a working solution, but we don't know yet how often it is "correct".

We can evaluate the output quality of the LLM in different ways, using both *offline* and *online* metrics. In our current situation, before we have brought the application to production, we are in the *offline* phase.

In this notebook, we will explore different forms of evaluations in the offline phase.

Some useful resources about LLM evals are [this blogpost](https://hamel.dev/blog/posts/evals/) and [this blogpost](https://medium.com/data-science-at-microsoft/evaluating-llm-systems-metrics-challenges-and-best-practices-664ac25be7e5).

## Unit testing basic functionality

We might not always have labeled data available. In any case, unit tests can help you build confidence in the system you're shipping to production.

We can test helper functions like we're already doing in `tests/test_extraction.py`. 

But we can also test the main functions that define our application, such as our `extract_general_info`, which extracts title and summary, or `extract_businesses_involved`, which extracts the businesses involved in the news article.

For example, we can assert that the summary is no more than 2 sentences, or that the extracted businesses indeed occur in the article.

What else could we test for?

In [1]:
from llmops_training.news_reader.data import get_bbc_news_sample
from llmops_training.news_reader.extraction import (
    extract_general_info,
    extract_businesses_involved,
    get_general_info_prompt_template,
    get_businesses_involved_prompt_template,
)

In [2]:
data = get_bbc_news_sample()
article = data[data["is_business"]].iloc[0].article

article

"Reality Check: Is lack of cash making women work past 70? - BBC News\n\nThe claim: The increase in women working in their 70s is because some of them can't afford to retire.\nReality Check verdict: Although some women keep working out of choice, it is also likely that others are doing so because increased life expectancy and an inadequate pension pot means they don't have enough money to retire on.\nThe proportion of women working into their 70s has doubled in the past four years, to 11%, according to official figures.\nThat works out as about 150,000 women still working into their mid-70s.\nAlthough the growth has been strong, there is still a higher proportion of men working into their 70s, at 15.5%.\nBut are some women continuing to work later in life because they want to, or because they cannot afford to retire?\nLife expectancy has been steadily climbing in the UK, and a woman who was 65 in 2015 could expect to live a further 20.9 years, on average, according to the Office for Na

> **Exercise** 📝
>
> - Fill in the TODO's in the test functions below.
> - Run the tests in the subsequent cell
> - Move these tests to the `tests/test_extraction.py` file
> - Run the tests from the command line using `uv run pytest tests`
> 


In [3]:
def test_summary_not_too_long(article: str):
    summary = extract_general_info(get_general_info_prompt_template(), article).summary
    summary_length = len(summary.split(". "))
    assert summary_length <= 2, f"Summary is too long: {summary_length} sentences"


def test_extracted_businesses_are_in_article(article: str):
    businesses = extract_businesses_involved(
        get_businesses_involved_prompt_template(), article
    ).businesses
    for business in businesses:
        # TODO: Fill me in! Assert that the business is in the article
        assert business in article, f"Business '{business}' not found in the article"

In [4]:
# %load ../solutions/evaluating-the-solution/unit-tests.py

In [5]:
test_summary_not_too_long(article)
test_extracted_businesses_are_in_article(article)

print("🎉 Tests passed!")

🎉 Tests passed!


## Evaluating using labeled data

To evaluate your application in more depth in a structured manner, it is essential to collect labeled data for your use case.

We can then define metrics, such as accuracy or ROUGE score, that we can iterate and improve upon.

Luckily, we already have some labeled examples that we can use for evaluating. We only load a few in this training, so the evaluations remain cheap and go relatively quick.

In [6]:
from typing import List, Optional

import pandas as pd

from llmops_training.news_reader.data import get_evaluation_data
from llmops_training.news_reader.extraction import (
    GeneralInfo, 
    extract_business_category_from_articles,
    get_business_category_prompt_template
)

In [7]:
# A few articles for illustration
eval_data = get_evaluation_data()
eval_data

,article,is_business,description,title
0,Tottenham Hotspur 6-0 Millwall - BBC Sport\n\n...,False,Son Heung-min scores a hat-trick as Tottenham ...,Tottenham Hotspur 6-0 Millwall - BBC Sport
1,Why transgender Africans turned against a famo...,False,Nigeria's LGBT community respond to transgende...,Why transgender Africans turned against a famo...
2,FA Cup: Chelsea 1-0 Manchester United - Funnie...,False,"Funnies, action and analysis from Chelsea's 1-...",FA Cup: Chelsea 1-0 Manchester United - Funnie...
3,Dubai Championships: Roger Federer loses to qu...,False,Australian Open champion Roger Federer suffers...,Dubai Championships: Roger Federer loses to qu...
4,"Scotland: Stuart Armstrong, Ryan Fraser & Tom ...",False,In-form Celtic midfielder Stuart Armstrong is ...,"Scotland: Stuart Armstrong, Ryan Fraser & Tom ..."
5,What makes gambling wrong but insurance right?...,True,Gamblers and insurers both place bets on the f...,What makes gambling wrong but insurance right?...
6,US ranchers saddle up for trade battle with Wa...,True,America's rural heavily supported Donald Trump...,US ranchers saddle up for trade battle with Wa...
7,How an obscure seed is helping to save the ele...,True,A look at how the seed of a South American tre...,How an obscure seed is helping to save the ele...
8,Shipping slump: Why a vessel worth $60m was so...,True,"Why are huge container ships, once worth milli...",Shipping slump: Why a vessel worth $60m was so...
9,Reality Check: Do 60% benefit from the changes...,True,Who are the winners and losers from the Budget...,Reality Check: Do 60% benefit from the changes...


We can evaluate different aspects of our application, such as:
- How many articles are correctly classified as *business* news or not
- How many of the titles are correctly extracted
- How many times the pipeline succeeds or errors
- How well the summaries corresponds to the labeled summaries

Let's implement the first one!

First, we classify for each article if it is business news or not. We can then compare this to the labeled data:

In [8]:
business_category_list = extract_business_category_from_articles(
    get_business_category_prompt_template(),
    eval_data["article"].tolist()
)
business_category_list

[BusinessCategory(is_about_business=False),
 BusinessCategory(is_about_business=False),
 BusinessCategory(is_about_business=False),
 BusinessCategory(is_about_business=False),
 BusinessCategory(is_about_business=False),
 BusinessCategory(is_about_business=True),
 BusinessCategory(is_about_business=True),
 BusinessCategory(is_about_business=True),
 BusinessCategory(is_about_business=True),
 BusinessCategory(is_about_business=True)]

> **Exercise** 📝
>
> - Inspect the `evaluate_business_classification` function below.
> - Fill in the TODO's in the function, so it accurately computes the accuracy of the business classification.
> - Inspect the `evaluation` module in our package, which already has some evaluation functions implemented.
> - Move this function to the `evaluation` module in our package.
> - Uncomment the TODO's in the `evaluation` module to include this evaluation function.
> - Run the `evaluation` module as script from the command line using:
>   ```bash
>   uv run python -m llmops_training.news_reader.evaluation
>   ```
> - Can you see the `business_classification_accuracy` in the output? 🎉

In [9]:
def evaluate_business_classification(
    business_category_list: List[Optional[GeneralInfo]], data: pd.DataFrame
) -> float:
    """Return accuracy of classification of whether an article is about business.

    Data should contain:
    - a column "is_business" with a boolean indicating whether the article is about business.

    The provided list of business categories should be in the same order as the data.
    """

    correct = 0
    incorrect = 0
    for i, row in data.iterrows():
        if business_category_list[i] is None:
            continue

        is_about_business_prediction = business_category_list[i].is_about_business
        is_about_business_labeled = row["is_business"]
        
        if is_about_business_prediction == is_about_business_labeled:
            correct += 1
        else:
            incorrect += 1

    accuracy = correct / (correct + incorrect)
    return accuracy

In [10]:
# %load ../solutions/evaluating-the-solution/evaluate-business-classification.py

## (Bonus) Making evaluations part of tests

We can also include these evaluations in our test suite. For example, we can assert that the business classification accuracy is above a certain threshold. We can do so similarly for each metric.

> **Exercise** 📝
>
> - Inspect the `test_run_evaluation` function below.
> - Fill in the TODO's in the function, so it asserts that the metrics pass rates are satisfied.
> - Move this function to the `tests/test_evaluation.py` file.
> - Run the tests from the command line using `uv run pytest tests`
>
> 💡 Hint:
>
> - Choose sensible pass rates for the metrics. Don't be too strict for now, or your tests will fail quite often.
> - From the summarization metrics, focus just on `summarization_rouge_1`

In [ ]:
from llmops_training.news_reader.evaluation import run_evaluation

In [ ]:
def test_run_evaluation(evaluation_data: pd.DataFrame):
    results = run_evaluation(evaluation_data)

    # TODO: Fill me in! Choose pass rates for metrics and assert they are satisfied

In [ ]:
# %load ../solutions/evaluating-the-solution/evaluation-threshold.py

---